<a href="https://colab.research.google.com/github/bobby-he/Neural_Tangent_Kernel/blob/master/notebooks/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bobby-he/Neural_Tangent_Kernel.git

Cloning into 'Neural_Tangent_Kernel'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 179 (delta 74), reused 35 (delta 10), pack-reused 0
Receiving objects: 100% (179/179), 21.47 MiB | 9.77 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [0]:
from Neural_Tangent_Kernel.src.NTK_net import LinearNeuralTangentKernel, FourLayersNet, train_net, circle_transform, variance_est, cpu_tuple,\
                                              AnimationPlot_lsq, kernel_leastsq_update, kernel_mats

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

matplotlib_is_available = True
try:
  from matplotlib import pyplot as plt
except ImportError:
  print("Will skip plotting; matplotlib is not available.")
  matplotlib_is_available = False

# Data params
data_mean =4
data_stddev = 1.25

# ### Uncomment only one of these to define what data is actually sent to the Discriminator
(name, preprocess, d_input_func) = ("Raw data", lambda data: data, lambda x: x)
#(name, preprocess, d_input_func) = ("Data and variances", lambda data: decorate_with_diffs(data, 2.0), lambda x: x * 2)
#(name, preprocess, d_input_func) = ("Data and diffs", lambda data: decorate_with_diffs(data, 1.0), lambda x: x * 2)
#(name, preprocess, d_input_func) = ("Only 4 moments", lambda data: get_moments(data), lambda x: 4)


In [96]:
print("Using data [%s]" % (name))

# ##### DATA: Target data and generator input data

def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (n, 1)))  # Gaussian

def get_generator_input_sampler():
    return lambda n: torch.rand(n, 1)  # Uniform-dist data into generator, _NOT_ Gaussian

# ##### MODELS: Generator model and discriminator model

class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Generator, self).__init__()
        self.map1 = LinearNeuralTangentKernel(input_size, hidden_size, w_sig = 1)
        self.map2 = LinearNeuralTangentKernel(hidden_size, hidden_size, w_sig = np.sqrt(5))
        self.map3 = LinearNeuralTangentKernel(hidden_size, output_size, w_sig = np.sqrt(5))
        self.f = f

    def forward(self, x):
        x = self.map1(x)
        x = self.f(x)
        x = self.map2(x)
        x = self.f(x)
        x = self.map3(x)
        return x

class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Discriminator, self).__init__()
        self.map1 = LinearNeuralTangentKernel(input_size, hidden_size, w_sig = 1)
        self.map2 = LinearNeuralTangentKernel(hidden_size, hidden_size, w_sig = np.sqrt(10))
        self.map3 = LinearNeuralTangentKernel(hidden_size, output_size, w_sig = np.sqrt(10))
        self.f = f

    def forward(self, x):
        x = self.f(self.map1(x))
        x = self.f(self.map2(x))
        x = self.f(self.map3(x))
        return x

def extract(v):
    return v.data.storage().tolist()

def stats(d):
    return [np.mean(d), np.std(d)]

def trainlol():
    # Model parameters
    g_input_size = 1      # Random noise dimension coming into generator, per output vector
    g_hidden_size = 500     # Generator complexity
    g_output_size = 1     # Size of generated output vector
    d_input_size = 1  # Minibatch size - cardinality of distributions
    d_hidden_size = 500    # Discriminator complexity
    d_output_size = 1     # Single dimension for 'real' vs. 'fake' classification
    minibatch_size = 500

    d_learning_rate = 0.001
    g_learning_rate = 0.001
    sgd_momentum = 0.9

    num_epochs = 20000
    print_interval = 1000
    d_steps = 5
    g_steps = 1

    dfe, dre, ge = 0, 0, 0
    d_real_data, d_fake_data, g_fake_data = None, None, None

    discriminator_activation_function = torch.sigmoid
    generator_activation_function = torch.tanh

    d_sampler = get_distribution_sampler(data_mean, data_stddev)
    gi_sampler = get_generator_input_sampler()
    G = Generator(input_size=g_input_size,
                  hidden_size=g_hidden_size,
                  output_size=g_output_size,
                  f=generator_activation_function).cuda()
    D = Discriminator(input_size=d_input_func(d_input_size),
                      hidden_size=d_hidden_size,
                      output_size=d_output_size,
                      f=discriminator_activation_function).cuda()
    criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
    d_optimizer = optim.SGD(D.parameters(), lr=d_learning_rate, momentum=sgd_momentum)
    g_optimizer = optim.SGD(G.parameters(), lr=g_learning_rate, momentum=sgd_momentum)

    for epoch in range(num_epochs):
        for d_index in range(d_steps):
            # 1. Train D on real+fake
            D.zero_grad()

            #  1A: Train D on real
            d_real_data = Variable(d_sampler(minibatch_size)).cuda()
            d_real_decision = D(d_real_data)
            d_real_error = criterion(d_real_decision, Variable(0.95*torch.ones([minibatch_size,1])).cuda())  # ones = true
            d_real_error.backward() # compute/store gradients, but don't change params

            #  1B: Train D on fake
            d_gen_input = Variable(gi_sampler(minibatch_size)).cuda()
            d_fake_data = G(d_gen_input).detach()  # detach to avoid training G on these labels
            d_fake_decision = D(d_fake_data)
            d_fake_error = criterion(d_fake_decision, Variable(0.05*torch.ones([minibatch_size,1])).cuda())  # zeros = fake
            d_fake_error.backward()
            d_optimizer.step()     # Only optimizes D's parameters; changes based on stored gradients from backward()

            dre, dfe = extract(d_real_error)[0], extract(d_fake_error)[0]

        for g_index in range(g_steps):
            # 2. Train G on D's response (but DO NOT train D on these labels)
            G.zero_grad()

            gen_input = Variable(gi_sampler(minibatch_size)).cuda()
            g_fake_data = G(gen_input)
            dg_fake_decision = D(preprocess(g_fake_data))
            g_error = criterion(dg_fake_decision, Variable(0.95*torch.ones([minibatch_size,1])).cuda())  # Train G to pretend it's genuine

            g_error.backward()
            g_optimizer.step()  # Only optimizes G's parameters
            ge = extract(g_error)[0]

        if epoch % print_interval == 0:
            print("Epoch %s: Fake Dist (%s),  D (%s real_err, %s fake_err) G (%s err); Real Dist (%s)" %
                  (epoch, stats(extract(d_fake_data)), dre, dfe, ge, stats(extract(d_real_data))))
            plt.clf()
            values = extract(g_fake_data)
            plt.hist(values, bins=20)
            plt.xlabel('Value')
            plt.ylabel('Count')
            plt.title('Histogram of Generated Distribution')
            plt.grid(True)
            plt.show()
            
    gen_input = Variable(gi_sampler(5000)).cuda()
    g_fake_data = G(gen_input)
    
    if matplotlib_is_available:
        print("Plotting the generated distribution...")
        values = extract(g_fake_data)
        #print(" Values: %s" % (str(values)))
        plt.clf()
        plt.hist(values, bins=100)
        plt.xlabel('Value')
        plt.ylabel('Count')
        plt.title('Histogram of Generated Distribution')
        plt.grid(True)
        plt.show()


Using data [Raw data]


In [0]:
    g_input_size = 1      # Random noise dimension coming into generator, per output vector
    g_hidden_size = 500     # Generator complexity
    g_output_size = 1     # Size of generated output vector
    d_input_size = 1  # Minibatch size - cardinality of distributions
    d_hidden_size = 500    # Discriminator complexity
    d_output_size = 1     # Single dimension for 'real' vs. 'fake' classification
    minibatch_size = 500

    d_learning_rate = 0.001
    g_learning_rate = 0.001
    sgd_momentum = 0.9

    num_epochs = 20000
    print_interval = 1000
    d_steps = 5
    g_steps = 1

    dfe, dre, ge = 0, 0, 0
    d_real_data, d_fake_data, g_fake_data = None, None, None

    discriminator_activation_function = torch.sigmoid
    generator_activation_function = torch.tanh

    d_sampler = get_distribution_sampler(data_mean, data_stddev)
    gi_sampler = get_generator_input_sampler()
    G = Generator(input_size=g_input_size,
                  hidden_size=g_hidden_size,
                  output_size=g_output_size,
                  f=generator_activation_function).cuda()
    D = Discriminator(input_size=d_input_func(d_input_size),
                      hidden_size=d_hidden_size,
                      output_size=d_output_size,
                      f=discriminator_activation_function).cuda()
    criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
    d_optimizer = optim.SGD(D.parameters(), lr=d_learning_rate, momentum=sgd_momentum)
    g_optimizer = optim.SGD(G.parameters(), lr=g_learning_rate, momentum=sgd_momentum)


defaultdict(dict, {})